In [2]:
# # srednio to działa w jupyter notebooku
# import subprocess
# def build_graph(path_to_jar, path_to_gtfs):
#     subprocess.call(['java', '-Xmx2G', '-jar', path_to_jar, '--build', '--save', path_to_gtfs])

# def serve_graph(path_to_jar, path_to_graph_parent_dir):
#     subprocess.call(['java', '-Xmx2G', '-jar', path_to_jar, '--load', '--serve', path_to_graph_parent_dir])

In [3]:
# path_to_jar = '../otp/otp-2.2.0-shaded.jar'
# path_to_graph = '../otp/otp_graphs/2023-05-18.gtfs/'
# serve_graph(path_to_jar, path_to_graph)

In [3]:
import requests
from typing import Optional, Dict

def get_response(query_parameters: Optional[Dict[str, str]] = None) -> requests.Response:
    if query_parameters is None:
        query_parameters = {
            "fromPlace": "52.19487631988541,21.024312973022464",
            "toPlace": "52.228272576254064,20.96903800964356",
            "time": "10:49am",
            "date": "05-18-2023",
            "mode": "TRANSIT,WALK",
            "arriveBy": "false",
            "wheelchair": "false",
            "showIntermediateStops": "true",
            "debugItineraryFilter": "false",
            "walkReluctance": "10",
            "additionalParameters": "walkReluctance",
            "locale": "pl",
            "numItineraries": "10",
        }

    url = "http://localhost:8080/otp/routers/default/plan?"

    for key, value in query_parameters.items():
        url += f"{key}={value}&"

    # Remove the trailing "&" at the end of the URL
    url = url.rstrip("&")

    response = requests.get(url)
    return query_parameters, response

In [4]:
import pandas as pd
query, response = get_response(None)
itineraries_df = pd.DataFrame(response.json()['plan']['itineraries'])

### mapa najczęściej odwiedzanych węzłów przystanków

In [25]:
import folium
import pandas as pd

tmp = pd.read_csv('../dane/gtfs-drive/2023-05-18.gtfs/stop_times.txt')
tmp['stop_id'] = tmp['stop_id'].str.strip()
stops = pd.read_csv('../dane/gtfs-drive/2023-05-18.gtfs/stops.txt')
tmp = tmp.merge(stops, 'inner', on='stop_id')
tmp['node'] = tmp['stop_id'].apply(lambda x: ''.join(x.split('_')[:-1]))
tmp = tmp.groupby('node').count().sort_values(ascending=False, by='trip_id').reset_index(drop=False).merge(tmp, how='left', on='node')
tmp.drop_duplicates(subset='node', inplace=True)

# Create a map centered around the specified coordinates
center_lat, center_lon = 52.23, 21.01
my_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Define a color scale based on the 'trip_id_x' values using StepColormap
color_scale = folium.StepColormap(['green', 'yellow', 'red'],
                                   index=tmp['trip_id_x'].quantile([0, 0.33, 0.66, 1]).values,
                                   caption='Count')

# Add the color scale to the map
my_map.add_child(color_scale)

# Function to map numeric values to named colors
def get_color(value):
    if value < 100:
        return 'green'
    elif value < 2000:
        return 'lightred'
    else:
        return 'red'

# Add markers for each data point with colors based on 'trip_id_x'
for index, row in tmp[:150].iterrows():
    color = get_color(row['trip_id_x'])
    folium.Marker([row['stop_lat_y'], row['stop_lon_y']],
                  popup=f"{''.join(row['stop_name_y'].split(' ')[:-1])} - Count: {row['trip_id_x']}",
                  icon=folium.Icon(color=color)).add_to(my_map)

# Save the map as an HTML file
my_map.save("../plots/EDA/stops_occupancy.html")


# benchmark przesiadek

In [26]:
# muzeum wilanów : 52.165201167925254, 21.09049767259721
# pałac królewski stare miasto : 52.248963845311124, 21.01529146366968
# francuska : 52.234521294640146, 21.053933021267078
# arkadia : 52.2564580848624, 20.98385681963401
# lotnisko : 52.16466233177019, 20.969516668530133
# mordor : 52.179312617041084, 21.003784302226087
# muzeum wódki : 52.255986733363315, 21.045979380575165
# muzeum powstania : 52.23374344918061, 20.981035021640107
# promenada : 52.23149250916612, 21.10963998004316
# pkin : 52.2319102250639, 21.006027190762005
# muzeum kopernika : 52.2418748527458, 21.028727101088748
# łazienki + belweder : 52.21321469405343, 21.027641805815158
# stadion narodowy : 52.23861443473494, 21.044607973740646
# wyścigi konne służew : 52.16066953299534, 21.013316804040734
# pole mokotowskie / politechnika : 52.212869749343454, 21.005889381059525
# sokowirówka wilanów : 52.15877563478556, 21.072135559952702
# powązki : 52.25279912233613, 20.981064442717255

In [43]:
origins = ['52.165201167925254, 21.09049767259721', '52.234521294640146, 21.053933021267078']
destinations = ['52.248963845311124, 21.01529146366968', '52.16066953299534, 21.013316804040734']
time = '08:00am'
date = '05-18-2023'
df = pd.DataFrame()
for src, dst in zip(origins, destinations):
    query_parameters = {
            "fromPlace": src,
            "toPlace": dst,
            "time": time,
            "date": date,
            "mode": "TRANSIT,WALK",
            "arriveBy": "false",
            "wheelchair": "false",
            "showIntermediateStops": "true",
            "debugItineraryFilter": "false",
            "walkReluctance": "10",
            "maxWalkDistance" : '1000',
            "additionalParameters": "walkReluctance, maxWalkDistance",
            "locale": "pl",
            "numItineraries": "10",
        }
    q, response = get_response(query_parameters)

    df = pd.concat([df, pd.json_normalize(response.json())], ignore_index=True)
df = df.drop(columns = ['previousPageCursor',
                        'nextPageCursor',
                        'elevationMetadata.ellipsoidToGeoidDifference',
                        'elevationMetadata.geoidElevation',
                        'requestParameters.maxWalkDistance',
                        'requestParameters.locale',
                        'requestParameters.walkReluctance',
                        'requestParameters.numItineraries',
                        'requestParameters.arriveBy',
                        'requestParameters.debugItineraryFilter',
                        'requestParameters.showIntermediateStops',
                        'requestParameters.wheelchair',
                        'requestParameters.fromPlace',
                        'requestParameters.mode',
                        'requestParameters.toPlace',
                        'requestParameters.additionalParameters',
                        'plan.date',
                        'plan.from.name',
                        'plan.from.vertexType',
                        'plan.to.name',
                        'plan.to.vertexType',
                        'metadata.searchWindowUsed',
                        'metadata.nextDateTime',
                        'metadata.prevDateTime',
                        'debugOutput.precalculationTime',	
                        'debugOutput.directStreetRouterTime',
                        'debugOutput.transitRouterTime',	
                        'debugOutput.filteringTime',	
                        'debugOutput.renderingTime',	
                        'debugOutput.totalTime',	
                        'debugOutput.transitRouterTimes.tripPatternFilterTime',
                        'debugOutput.transitRouterTimes.accessEgressTime',
                        'debugOutput.transitRouterTimes.raptorSearchTime',
                        'debugOutput.transitRouterTimes.itineraryCreationTime',
                        ])  
df = df.explode('plan.itineraries').reset_index().rename(columns={'plan.itineraries' : 'itinerary'})
df.rename(columns={'index' : 'route_nr'}, inplace=True)
print(df.shape)
df.head()

(19, 8)


,route_nr,requestParameters.date,requestParameters.time,plan.from.lon,plan.from.lat,plan.to.lon,plan.to.lat,itinerary
0,0,05-18-2023,08:00am,21.090498,52.165201,21.015292,52.248964,"{'duration': 3287, 'startTime': 1684390135000,..."
1,0,05-18-2023,08:00am,21.090498,52.165201,21.015292,52.248964,"{'duration': 3263, 'startTime': 1684390195000,..."
2,0,05-18-2023,08:00am,21.090498,52.165201,21.015292,52.248964,"{'duration': 3227, 'startTime': 1684390555000,..."
3,0,05-18-2023,08:00am,21.090498,52.165201,21.015292,52.248964,"{'duration': 3287, 'startTime': 1684390735000,..."
4,0,05-18-2023,08:00am,21.090498,52.165201,21.015292,52.248964,"{'duration': 3623, 'startTime': 1684390735000,..."


In [46]:
# rozkładanie na pojedyncze trasy
tmp = pd.DataFrame()
for num, row in df.iterrows():
    tmp = pd.concat([tmp, pd.json_normalize(row['itinerary'])])
tmp = tmp.reset_index()
df = df.merge(tmp, left_index=True, right_index=True)
df.drop('itinerary', axis = 1, inplace = True)
df.rename(columns = {'index_x' : 'queryNumber',
                            'index_y' : 'itinerary'})
df.reset_index(drop=False).rename(columns={'index':'itinerary_nr'}, inplace=True)

In [14]:
# rozkładanie na pojedyncze "legs"
tmp = pd.DataFrame()
for num, row in df.iterrows():
    leg = pd.json_normalize(row['legs'])
    leg['route_nr'] = row['route_nr']
    tmp = pd.concat([tmp, ])
tmp = tmp.reset_index().rename(columns={'index' : 'leg_nr'})
# df = df.merge(tmp, left_index=True, right_index=True)
# df.drop('legs', axis = 1, inplace = True)
# df
tmp

,index,startTime,endTime,departureDelay,arrivalDelay,realTime,distance,generalizedCost,pathway,mode,...,serviceDate,intermediateStops,routeShortName,routeLongName,from.stopId,from.arrival,from.stopIndex,from.stopSequence,to.stopIndex,to.stopSequence
0,0,1684390135000,1684390620000,0,0,False,579.81,4385,False,WALK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1684390620000,1684392720000,0,0,False,10740.04,2700,False,BUS,...,2023-05-18,"[{'name': 'Królowej Marysieńki 02', 'stopId': ...",116,Chomiczówka,1:3044_02,1.684391e+12,1.0,2.0,23.0,26.0
2,2,1684392720000,1684393422000,0,0,False,718.38,5384,False,WALK,...,NaN,NaN,NaN,NaN,1:7048_02,1.684393e+12,NaN,NaN,NaN,NaN
3,0,1684390195000,1684390680000,0,0,False,579.81,4385,False,WALK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1684390680000,1684391160000,0,0,False,2647.81,1080,False,BUS,...,2023-05-18,"[{'name': 'Królowej Marysieńki 02', 'stopId': ...",164,Wilanów,1:3044_02,1.684391e+12,11.0,14.0,16.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,1,1684393020000,1684394640000,0,0,False,8420.48,2220,False,BUS,...,2023-05-18,"[{'name': 'Katowicka 01', 'stopId': '1:2154_01...",138,Utrata,1:2153_01,1.684393e+12,19.0,23.0,38.0,43.0
94,2,1684394820000,1684395240000,0,0,False,1980.12,1200,False,BUS,...,2023-05-18,"[{'name': 'rondo Unii Europejskiej 01', 'stopI...",218,Królikarnia,1:3242_01,1.684395e+12,14.0,16.0,19.0,21.0
95,3,1684395240000,1684395285000,0,0,False,34.21,272,False,WALK,...,NaN,NaN,NaN,NaN,1:3009_16,1.684395e+12,NaN,NaN,NaN,NaN
96,4,1684395780000,1684396140000,0,0,False,1801.44,1455,False,BUS,...,2023-05-18,"[{'name': 'al.Lotników 01', 'stopId': '1:3011_...",192,Metro Wilanowska,1:3009_17,1.684395e+12,0.0,1.0,2.0,3.0
